# Semi-Supervisée

In [1]:
import numpy as np 
np.set_printoptions(threshold=10000,suppress=True) 
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

# I. Découpage de la base en apprentissage/test

In [2]:
data = pd.read_csv('./Wave.txt', delimiter=' ', header=None)
y = data[40]
data = data.drop(columns=[40])
print('labels')
print(y)
data

labels
0       2
1       0
2       1
3       0
4       1
       ..
4995    0
4996    1
4997    2
4998    0
4999    0
Name: 40, Length: 5000, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-0.23,-1.21,1.20,1.23,-0.10,0.12,2.49,1.19,1.34,0.58,...,-0.63,-0.86,-0.70,0.51,0.34,-0.13,-0.87,0.56,-0.53,0.29
1,0.38,0.38,-0.31,-0.09,1.52,1.35,1.49,3.81,2.33,1.34,...,0.31,1.28,1.40,0.00,-0.18,0.71,0.04,0.91,-0.79,0.22
2,-0.69,1.00,1.08,1.48,2.44,3.39,3.09,4.08,5.48,3.61,...,0.93,0.29,1.12,0.60,0.28,2.17,0.18,-0.09,-1.33,1.00
3,0.40,0.68,0.27,1.39,1.03,-0.32,-1.23,-0.50,0.11,0.87,...,1.18,0.43,-0.30,-0.07,-0.99,-0.75,1.11,1.35,-1.63,0.10
4,-0.81,1.59,-0.69,1.16,4.22,4.98,4.52,2.54,5.60,4.66,...,-0.07,0.62,0.14,1.33,-1.87,1.48,-0.02,-0.58,0.93,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.44,0.56,1.84,1.94,3.43,4.88,4.04,2.11,1.83,0.78,...,0.52,-2.56,-0.68,0.97,-0.78,-1.14,0.45,0.18,1.44,1.11
4996,1.18,-0.48,1.81,1.51,1.41,3.61,3.75,3.80,3.44,3.71,...,-0.41,0.37,-1.96,-1.19,-1.08,1.37,-1.02,0.71,-0.10,0.36
4997,0.64,0.81,-0.38,-0.88,1.55,0.10,0.42,-0.93,-0.70,1.54,...,0.49,1.61,-0.81,-1.35,-0.84,-0.64,1.19,-0.38,-0.70,-0.85
4998,0.18,1.65,1.91,2.07,4.28,3.61,4.46,4.62,4.80,0.25,...,0.17,-0.65,-0.73,0.89,-0.97,-1.30,-0.20,-0.63,-0.92,0.63


In [10]:
np.random.seed(123)
X_A, X_T, y_A , y_T = train_test_split(data, y, train_size=0.5, test_size=0.5)

# II. Simulation de l’aspect semi-supervisé

In [ ]:
def remove_labels(labels: pd.DataFrame, taux: int = 50):
    labels_cp = labels.copy()
    index = np.array(labels_cp.index)
    np.random.shuffle(index)
    nb_to_select = int((len(index)*taux)/100)
    print(nb_to_select)
    print(len(index))
    index = index[:nb_to_select]
    for i in index:
        labels_cp.loc[index] = -1
    return labels_cp
y_A_semi = remove_labels(y_A, 5)
y_A_semi

125
2500


3721    1
1904    2
3563    0
4034    0
2199    0
       ..
4295    1
2400    2
325     1
266     0
1468    1
Name: 40, Length: 2500, dtype: int64

# III. Sélection de variables semi-supervisée

In [16]:
var_no_lab = X_A[0][y_A_semi == -1]
var_no_lab

1175   -0.87
56     -0.59
2735   -0.03
1609    1.15
4580   -0.33
        ... 
255    -0.10
371     0.96
4309    0.28
3481    0.85
4169   -0.02
Name: 0, Length: 125, dtype: float64

In [ ]:
def s1(variable, y_A_semi): # Label
    sum_haut = 0
    sum_bas = 0
    for i in y_A_semi.unique():
        n_i = len(y_A_semi.where(y_A_semi == i).dropna())
        u_i = np.mean(variable.where(y_A_semi == i).dropna())
        u = np.mean(variable)
        sum_haut += n_i*(u_i-u)**2
        #print(sum_haut)

        o_i = np.std(variable.where(y_A_semi == i).dropna())
        sum_bas += n_i*o_i**2
    return sum_haut/sum_bas

def s2(variable):
    sum_haut = 0
    for i, v_i in enumerate(variable):
        for j, v_j in enumerate(variable):
            if i != j:
                S_ij = np.exp(-np.linalg.norm(X_A.iloc[i]-X_A.iloc[j])**2/10)
                sum_haut += (v_i-v_j)**2 * S_ij
    print("aaaaaaaaaaa", S_ij)
    print("ccc", np.var(variable))
    return sum_haut / np.var(variable)

def score(v):
    y_A_lab = y_A_semi.where(y_A_semi != -1).dropna()
    y_A_no_lab = y_A_semi.where(y_A_semi == -1).dropna()
    var_no_lab = v.loc[np.array(y_A_no_lab.index)]
    s1_res = s1(v, y_A_lab)
    s2_res = s2(var_no_lab)
    #print(s2_res)
    return s1_res / s2_res

for i, v in enumerate(X_A.columns):
    print('#############',i, '#################')
    print(score(X_A[v]))

############# 0 #################
aaaaaaaaaaa 1.0
ccc 1.0526388863999996
0.00011285304813691767
############# 1 #################
aaaaaaaaaaa 1.0
ccc 0.8752708096
9.596149126096535e-05
############# 2 #################
aaaaaaaaaaa 1.0
ccc 1.3536505983999998
0.00010158714985101602
############# 3 #################
aaaaaaaaaaa 1.0
ccc 1.9845040384
0.00032039531843296013
############# 4 #################
aaaaaaaaaaa 1.0
ccc 2.9607755904000004
3.7066391408594515e-05
############# 5 #################
aaaaaaaaaaa 1.0
ccc 3.3980367743999995
9.034761707983282e-05
############# 6 #################
aaaaaaaaaaa 1.0
ccc 4.247255039999999
0.0001636993049944407
############# 7 #################
aaaaaaaaaaa 1.0
ccc 3.2284694399999996
0.0001750905641981278
############# 8 #################
aaaaaaaaaaa 1.0
ccc 2.58916864
0.00016391561696922196
############# 9 #################
aaaaaaaaaaa 1.0
ccc 1.9345714304
0.00016558757573887487
############# 10 #################
aaaaaaaaaaa 1.0
ccc 2.83181373440000

In [ ]:
def plot_variable_scores(scores : list[float]):
    plt.figure(figsize=(10, 6))
    plt.bar(range(1, len(scores) + 1), scores)
    plt.xlabel("Variables")
    plt.ylabel("Score de pertinence")
    plt.title("Histogramme des scores de pertinence des variables")
    plt.show()

# IV. Evaluation de la sélection